In [1]:
### 여의도역 근처 top5 아파트 시세
# 광장, 미성, 롯데캐슬엠파이어, 수정, 여의도자이

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [4]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [5]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [5]:
# 억단위로
def parse_korean_price(text):
    """
    예: '12억 5,000' → 12.5, '9억' → 9.0, '3억8000' → 3.8
    """
    if not isinstance(text, str) or text.strip() == "":
        return None

    text = text.replace(",", "").replace(" ", "")
    eok = 0
    man = 0

    if "억" in text:
        parts = text.split("억")
        eok = int(parts[0]) if parts[0].isdigit() else 0
        if len(parts) > 1 and parts[1].isdigit():
            man = int(parts[1])
    elif text.isdigit():
        man = int(text)

    return round(eok + man / 10000, 3)


In [6]:
## 광장
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=955"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['755369543174542955']


In [ ]:
## 광장
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=955&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest", # 이 요청이 자바스크립트를 통해 비동기로 보낸 것임을 나타냄 (AJAX 요청)
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]: # 각 달 별 데이터 반복
        yyyymm = monthly["yyyymm"] 
        for daily in monthly["val"]: # 달의 날짜 별로 반복
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)

key = '755369543174542955'

df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key)) # x를 입력받아 decrypt(x,key)로 반환
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))
df_1['가격(억단위)'] = df_1['가격'].apply(parse_korean_price)
df_1["연도"] = df_1["거래일"].apply(lambda x: int(x.split(".")[0]))
df_1["아파트명"] = "광장아파트"


df_1

,거래일,층수,가격,전월세,가격(억단위),연도,아파트명
0,2025.07.15,4,"32억 7,000",,32.70,2025,광장아파트
1,2025.07.09,1,32억,,32.00,2025,광장아파트
2,2025.06.27,1,29억,,29.00,2025,광장아파트
3,2025.06.23,9,36억,,36.00,2025,광장아파트
4,2025.06.23,2,28억,,28.00,2025,광장아파트
...,...,...,...,...,...,...,...
332,2006.01.21,6,"9억 9,800",,9.98,2006,광장아파트
333,2006.01.15,6,10억 500,,10.05,2006,광장아파트
334,2006.01.12,6,7억,,7.00,2006,광장아파트
335,2006.01.12,5,9억,,9.00,2006,광장아파트


In [7]:
## 미성
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=959"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['759369543174394959']


In [13]:
## 미성
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=959&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '759369543174394959'

df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))
df_2['가격(억단위)'] = df_2['가격'].apply(parse_korean_price)
df_2["연도"] = df_2["거래일"].apply(lambda x: int(x.split(".")[0]))
df_2["아파트명"] = "미성아파트"
df_2

,거래일,층수,가격,전월세,가격(억단위),연도,아파트명
0,2025.06.12,2,"26억 9,000",,26.90,2025,미성아파트
1,2025.05.17,6,"23억 7,000",,23.70,2025,미성아파트
2,2025.04.03,8,22억,,22.00,2025,미성아파트
3,2025.03.26,6,25억,,25.00,2025,미성아파트
4,2025.03.25,10,"23억 5,000",,23.50,2025,미성아파트
...,...,...,...,...,...,...,...
297,2006.01.22,3,"5억 8,300",,5.83,2006,미성아파트
298,2006.01.16,6,"12억 2,000",,12.20,2006,미성아파트
299,2006.01.13,12,"8억 5,000",,8.50,2006,미성아파트
300,2006.01.12,6,6억 500,,6.05,2006,미성아파트


In [9]:
## 롯데캐슬엠파이어
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=617938"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['758369520346171617938']


In [14]:
## 롯데캐슬엠파이어
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=617938&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '758369520346171617938'

df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))
df_3['가격(억단위)'] = df_3['가격'].apply(parse_korean_price)
df_3["연도"] = df_3["거래일"].apply(lambda x: int(x.split(".")[0]))
df_3["아파트명"] = "롯데캐슬엠파이어"
df_3

,거래일,층수,가격,전월세,가격(억단위),연도,아파트명
0,2025.06.24,14,"22억 2,000",,22.2,2025,롯데캐슬엠파이어
1,2025.06.20,31,30억,,30.0,2025,롯데캐슬엠파이어
2,2025.06.19,3,24억,,24.0,2025,롯데캐슬엠파이어
3,2025.06.14,30,"26억 8,000",,26.8,2025,롯데캐슬엠파이어
4,2025.06.10,5,21억,,21.0,2025,롯데캐슬엠파이어
...,...,...,...,...,...,...,...
274,2006.02.16,11,12억,,12.0,2006,롯데캐슬엠파이어
275,2006.02.07,32,13억,,13.0,2006,롯데캐슬엠파이어
276,2006.02.02,27,"12억 4,000",,12.4,2006,롯데캐슬엠파이어
277,2006.01.19,20,"8억 7,000",,8.7,2006,롯데캐슬엠파이어


In [11]:
## 수정
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=964"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['754369543174209964']


In [15]:
## 수정
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=964&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=50&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '754369543174209964'

df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))
df_4['가격(억단위)'] = df_4['가격'].apply(parse_korean_price)
df_4["연도"] = df_4["거래일"].apply(lambda x: int(x.split(".")[0]))
df_4["아파트명"] = "수정아파트"
df_4

,거래일,층수,가격,전월세,가격(억단위),연도,아파트명
0,2020.06.13,10,"20억 5,000",,20.5,2020,수정아파트
1,2020.06.13,2,20억,,20.0,2020,수정아파트
2,2020.06.12,3,20억,,20.0,2020,수정아파트
3,2020.04.30,14,19억,,19.0,2020,수정아파트
4,2019.12.13,1,"20억 3,000",,20.3,2019,수정아파트
...,...,...,...,...,...,...,...
175,2006.03.08,7,"6억 7,000",,6.7,2006,수정아파트
176,2006.03.06,3,"12억 9,000",,12.9,2006,수정아파트
177,2006.03.01,9,"12억 5,000",,12.5,2006,수정아파트
178,2006.02.28,14,"6억 8,000",,6.8,2006,수정아파트


In [13]:
## 여의도자이
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=20090966"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75636879984413520090966']


In [16]:
## 여의도자이
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=20090966&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '75636879984413520090966'

df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))
df_5['가격(억단위)'] = df_5['가격'].apply(parse_korean_price)
df_5["연도"] = df_5["거래일"].apply(lambda x: int(x.split(".")[0]))
df_5["아파트명"] = "여의도자이"
df_5

,거래일,층수,가격,전월세,가격(억단위),연도,아파트명
0,2025.06.23,18,"35억 5,000",,35.50,2025,여의도자이
1,2025.06.12,3,"39억 5,000",,39.50,2025,여의도자이
2,2025.06.10,25,"32억 5,000",,32.50,2025,여의도자이
3,2025.06.10,15,"33억 6,000",,33.60,2025,여의도자이
4,2025.05.15,30,"36억 8,000",,36.80,2025,여의도자이
...,...,...,...,...,...,...,...
324,2008.07.25,34,"15억 9,500",,15.95,2008,여의도자이
325,2008.07.17,12,17억,,17.00,2008,여의도자이
326,2008.06.25,6,"15억 1,000",,15.10,2008,여의도자이
327,2008.06.13,4,"14억 8,000",,14.80,2008,여의도자이
